In [1]:
import Minkowski as mn
import AdS as ads
from mm_estimator import mm_dimension
from longest_chain import *
from interval import filter_interval
import matplotlib.pyplot as plt
import numpy as np
%matplotlib tk
from importlib import reload

params= {
   'axes.labelsize': 16,
   'font.size': 18,
   'legend.fontsize': 18,
   'xtick.labelsize': 16,
   'ytick.labelsize': 16,
   'figure.figsize': [6, 6] 
   }
plt.rcParams.update(params)

def scatter(position,x=0,y=1):
    plt.scatter(position[:,x],position[:,y])

def plot_interval(l,interval_centre,half=True):
   R=interval_centre[0]
   t=interval_centre[1]
   if half==True:
      plt.plot([R,R+l/2,R],[t+l/2,t,t-l/2])
   else:
      plt.plot([R,R+l/2,R,R-l/2,R],[t+l/2,t,t-l/2,t,t+l/2])

def plot_link(position,link,xindex=0,yindex=1):
    for i in link:
        for j in link[i]:
            plt.plot([position[i,xindex],position[j,xindex]],[position[i,yindex],position[j,yindex]])

In [26]:
N=100
L=1
d=2
interval_centre=np.zeros(d)
position=mn.sprinkle(N,L,d,False)
link=find_link(position)
# position_subset=filter_interval(position,L,interval_centre)
scatter(position)
plot_link(position,link)
# scatter(position_subset)
plt.show()

In [14]:
#Sprinkle into AdS space
N=1000
L=1
d=2
R_0=0.2
interval_centre=[R_0,0]
for i in range(d-2):
    interval_centre.append(0)
position=ads.sprinkle(N,L,d,R_0,False,False)
position_subset=filter_interval(position,L,interval_centre)
plt.title(r'Sprinkling into AdS space at N=%s L=%s d=%s $R_0$=%s' %(N,L,d,R_0),fontsize=15)
plt.xlabel('R')
plt.ylabel('t')
scatter(position)
scatter(position_subset)
plot_interval(L,interval_centre,True)
plt.show()